## OpenBCI EEG data analysis (reading vs writing state)
Based on implementation by Viacheslav Nesterov (https://github.com/Vyachez/Project_BCI)

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from numpy import argmax
import seaborn as sns

from imp import reload

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


#### Variable and function definitions

In [2]:
# DEFINITIONS
path_r = 'openbci-data/OpenBCI-RAW-2019-02-19_23-19-58-read.txt'
path_w = 'openbci-data/OpenBCI-RAW-2019-02-19_23-23-02-write.txt'
dur_model = 990
secbatch = 10  
margin = 15000                    

In [3]:
def prepare_timeseries(dset):
    dset = dset.drop(columns=[0,9,10,11,12])
    dset = dset.reset_index(drop=True)
    dset.index=pd.to_datetime(dset[13],unit='ms')
    return dset

In [4]:
def upsample(dset, methodname):
    dset.index = pd.to_datetime(dset[13],unit='ms')
    upsampled = dset.resample('900000ns').mean().interpolate(method=methodname)
    return upsampled

In [5]:
def clean_upsampled(dset):
    dset['time'] = dset[13]
    dset['sec']  = dset.index
    dset['sec']  = dset['sec'].astype(str).str[11:-7]
    dset = dset.drop(columns=[13])
    dset = dset.reset_index(drop=True)
    dset = dset.drop(columns=['time'])
    dset = dset.reset_index(drop=True)
    return dset

In [6]:
def balance_check(dset, dur):
    for i in np.unique(dset['sec']):
        if len(dset['sec'].loc[dset['sec'] == i]) != dur:
            print('Seconds are not equal!')
            print(np.array([i, len(dset['sec'].loc[dset['sec'] == i])]))
    print("Check completed for balanced intervals!")
    return True

In [7]:
# balancing intervals to set length (optimal number of rows)
def balance_intervals(dset, int_no):
    """ dset - dataset to cleanup
        int_no - min length of intervals within one second"""
    idarr = np.array([[i, len(dset['sec'].loc[dset['sec'] == i])] for i in np.unique(dset['sec'])])
    idarr[0:3]
    for i in idarr:
        if int(i[1]) < int_no:
            date = i[0]
            # removing short/incomplete
            dset = dset.drop(index=dset.loc[dset['sec'] == date].index)
        elif int(i[1]) > int_no:
            date = i[0]
            end_ind = dset.loc[dset['sec'] == date].index[-1]
            cut_ind = dset.loc[dset['sec'] == date].index[int_no-1]
            # cutting excessive
            dset = dset.drop(dset.index[cut_ind:end_ind])
        dset = dset.reset_index(drop=True)
    return dset


In [8]:
# see if all of intervals have the same length by second
def check_sec_int(dset):
    return np.array([[i, len(dset['sec'].loc[dset['sec'] == i])] for i in np.unique(dset['sec'])])


In [9]:
def variance_clean(dset, var):
    """ dset - dataset to cleanup
        var - max variance - all that above is to catch and remove
    """
    for chan in range(8):
        for sec in np.unique(dset['sec']):
            min_edge = min(dset.loc[dset['sec'] == sec][chan+1])
            max_edge = max(dset.loc[dset['sec'] == sec][chan+1])
            variance = max_edge - min_edge
            idx = dset.loc[dset[chan+1] == max_edge].index[0]
            #print(variance)
            if variance > var:
                print('Channel {} | Second {} | Index {} | Variance:] = {}'.format(chan+1, sec, idx, variance))
                dset = dset.drop(index=dset.loc[dset['sec'] == sec].index)
                # reseting the index
                dset = dset.reset_index(drop=True)
                print('Dropped')
    print('Cleaned spikes larger than', var)
    return dset   


In [10]:
def seconds(dset):
    '''takes dataset as a  argument and utputs
    number of unique seconds'''
    print("\nSeconds in dataset now: ", len(np.unique(dset['sec'])))

In [11]:
# scaling function
def scaler(dset, secs, dur):
    '''Scaling function takes dataset of 8 channels
    and returns rescaled dataset.
    Rescales by interval (second).
    arg 'secs' is number of seconds to take into one scaling tensor.
    Scaling is between 1 and 0
    All datasets for training should be equalized'''
    # first - getting length of dataset modulo number of seconds for scaling
    intlv = secs*dur
    if len(dset['sec'])/intlv > 1:
        lendset = int(len(dset['sec'])/intlv)
        dset = dset[0:lendset*intlv]
        dset = dset.reset_index(drop=True)
    else:
        print("Inappropriate length of dataset. Too short. Set up less seconds for batch or choose another dataset.")
    seconds(dset)
    # now scaling
    if balance_check(dset, dur=dur):
        for chan in range(8):
            for i in range(int(len(dset['sec'])/intlv)):
                tmpdat = dset.loc[i*intlv:i*intlv+intlv-1, chan+1]
                tmpdat = (tmpdat-min(tmpdat))/(max(tmpdat)-min(tmpdat))  
                dset.loc[i*intlv:i*intlv+intlv-1, chan+1]= tmpdat
        dset = dset.reset_index(drop=True)
        print("Dataset has been rescaled \n")
        return dset
    else:
        print("\nDataset intervals are not balanced! Check the code and functions order.")


In [12]:
# data preprocessing function
def preprocess(fl, dur=dur_model, var=margin, secbatch=secbatch):
    '''"fl" (fullpath) - Takes dataset file
    "dur" - length of second - number of items per second
    "var" - variance of wave that need to be trimmed
    "secbatch" - batch of seconds to scale
    Returns balanced preprocessed data'''
    # loading the data
    dset = pd.read_csv(fl, sep=",", header=None)
    # convert dataframe into time series in order to upsample
    dset = prepare_timeseries(dset)
    # upsample the data
    dset = upsample(dset,'linear')
    # clean upsampled data
    dset = clean_upsampled(dset)
    # checking length
    seconds(dset)
    # cleaning spikes
    dset = variance_clean(dset, var)
    # checking length again
    seconds(dset)
    # balancing seconds intervals to have same length
    dset = balance_intervals(dset, dur)
    balance_check(dset, dur)
    # trimming and scaling to fit the integrity for data prep
    dset = scaler(dset, secbatch, dur)
    return dset

In [13]:
# Full 8 channels conversion
# each second to be converted in one n-dimentional array
def conversion_8(df):
    dat_lst = []
    for s in np.unique(df['sec']):
        dat_lst.append(np.array(df[[1,2,3,4,5,6,7,8]].loc[df['sec'] == s]))
    return np.array(dat_lst, dtype=float)

#### Neural Network Model 

In [14]:
# model architecture
num_classes = 2
model = Sequential()
model.add(Conv1D(32, kernel_size=(3), padding='same',
                 activation='relu',
                 input_shape=(990, 8,)))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling1D(pool_size=5, padding='same'))
model.add(Conv1D(64, (3), activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling1D(pool_size=3, padding='same'))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 990, 32)           800       
_________________________________________________________________
batch_normalization_1 (Batch (None, 990, 32)           3960      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 198, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 196, 64)           6208      
_________________________________________________________________
batch_normalization_2 (Batch (None, 196, 64)           784       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 66, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4224)              0         
__________

In [16]:
# Compile the model using a loss function and an optimizer.
adam = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

#### Load the Model with the Best Classification Accuracy

In [17]:
#model wights taken from (https://github.com/Vyachez/Project_BCI)
model.load_weights('r_w.weights.best.hdf5')

#### Loading and preprocessing data

In [18]:
# preprocessing
read_test = preprocess(path_r, dur=dur_model, var=margin, secbatch=secbatch)


Seconds in dataset now:  36
Cleaned spikes larger than 15000

Seconds in dataset now:  36
Check completed for balanced intervals!

Seconds in dataset now:  30
Check completed for balanced intervals!
Dataset has been rescaled 



In [19]:
# preprocessing
write_test = preprocess(path_w, dur=dur_model, var=margin, secbatch=secbatch)


Seconds in dataset now:  40
Cleaned spikes larger than 15000

Seconds in dataset now:  40
Check completed for balanced intervals!

Seconds in dataset now:  30
Check completed for balanced intervals!
Dataset has been rescaled 



In [20]:
r_dat_xt = conversion_8(read_test)
w_dat_xt = conversion_8(write_test)

#### Testing prediction

In [21]:
# predict any class
def predict_all(data, st):
    '''takes data - array with tensors for prediction
    predict any state depending on labels match '''
    timer = []
    sec = 0
    count_state = []
    for i in data:
        sec += 1
        count_all = timer.append(sec)
        if model.predict(np.array([i]))[0][0] > 0.5:
            count_state.append(0)
            #print('Second {}: {}'.format(sec, states[0]))
        elif model.predict(np.array([i]))[0][1] > 0.5:
            count_state.append(1)
            #print('Second {}: {}'.format(sec, states[1]))
        else:
            count_state.append(2)
            #print('Second {}: {}'.format('Unknown'))
                  
    read_count = np.count_nonzero(np.array(count_state) == 0)
    write_count = np.count_nonzero(np.array(count_state) == 1)
    unknown_count = np.count_nonzero(np.array(count_state) == 2)

    if st == 'r':
        print('\nPredicted reading: {} sec | unknown: {} sec | from total {} sec of READING set'.format(read_count,
                                                                                                     unknown_count,
                                                                                             len(count_state)))
        print('Prediction accuracy for READING on new environment dataset is: {}%'.format(int(read_count/len(count_state)*100)))
        st_int_r = 1
        st_int_w = 0
    elif st == 'w':
        print('\nPredicted writing: {} sec | unknown: {} sec | from total {} sec of WRITING set'.format(write_count,
                                                                                                     unknown_count,
                                                                                             len(count_state)))
        print('Prediction accuracy for WRITING on new environment dataset is: {}%'.format(int(write_count/len(count_state)*100)))
        st_int_r = 0
        st_int_w = 1
    return {st_int_r: int(read_count/len(count_state)*100), st_int_w: int(write_count/len(count_state)*100)}

In [22]:
benchmark = [predict_all(r_dat_xt, 'r'), predict_all(w_dat_xt, 'w')]
acc = []
for b in benchmark:
    acc.append(b[1])
print('\nReal model accuracy is: {}%'.format(np.average([acc])))


Predicted reading: 26 sec | unknown: 0 sec | from total 30 sec of READING set
Prediction accuracy for READING on new environment dataset is: 86%

Predicted writing: 3 sec | unknown: 0 sec | from total 30 sec of WRITING set
Prediction accuracy for WRITING on new environment dataset is: 10%

Real model accuracy is: 48.0%
